In [ ]:
! wget https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv

--2023-08-11 14:37:58--  https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv
Resolving static.openfoodfacts.org (static.openfoodfacts.org)... 213.36.253.206
Connecting to static.openfoodfacts.org (static.openfoodfacts.org)|213.36.253.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8386534133 (7.8G) [text/csv]
Saving to: ‘en.openfoodfacts.org.products.csv’

en.openfoodfacts.or 100%[===================>]   7.81G  18.5MB/s    in 7m 51s  

2023-08-11 14:45:50 (17.0 MB/s) - ‘en.openfoodfacts.org.products.csv’ saved [8386534133/8386534133]



In [ ]:
import pandas as pd

Extraction of the features I'll be using later.
After this I will clean the data. Just dropping each row would result in a great loss of data, so before I do that I decided to apply some filtering to the columns.

First of all, it seems that a lot of the products have an unknown type of packaging and packaging coordinates, so I'm just going to fill those values with the most common ones in the column

In [ ]:
cols_to_keep=["product_name","packaging","main_category",
              "nutrition-score-fr_100g","fat_100g","carbohydrates_100g","proteins_100g",
              "first_packaging_code_geo"]

alphabet = ['a', 'e', 'h']

df = pd.read_csv("/content/en.openfoodfacts.org.products.csv", sep="\t", header=0, usecols=cols_to_keep, low_memory=True)
world_food_data=df.rename(columns={"nutrition-score-fr_100g":"nutrition_score", \
                                                "fat_100g":"fat_g", \
                                                "carbohydrates_100g":"carbohydrates_g", \
                                                "proteins_100g":"proteins_g"})

most_common_coords=world_food_data.first_packaging_code_geo.value_counts().index[0]
most_common_packaging=world_food_data.packaging.value_counts().index[0]

world_food_data.first_packaging_code_geo.loc[world_food_data.first_packaging_code_geo.isnull()]=most_common_coords
world_food_data.packaging.loc[world_food_data.packaging.isnull()]=most_common_packaging

world_food_data=world_food_data.dropna()

world_food_data.to_csv('new_data1.csv')
print('LEN OF DATA - ', len(world_food_data))

<ipython-input-6-ed2eebb9dbb7>:7: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/en.openfoodfacts.org.products.csv", sep="\t", header=0, usecols=cols_to_keep, low_memory=True)
<ipython-input-6-ed2eebb9dbb7>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  world_food_data.first_packaging_code_geo.loc[world_food_data.first_packaging_code_geo.isnull()]=most_common_coords
<ipython-input-6-ed2eebb9dbb7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  world_food_data.packaging.loc[world_food_data.packaging.isnull()]=most_common_packaging


LEN OF DATA -  2948839


In [ ]:
world_food_data.head()

,product_name,packaging,first_packaging_code_geo,main_category,fat_g,carbohydrates_g,proteins_g,nutrition_score
3,Skyr,Plastique,"47.833333,-0.333333",en:cream-cheeses,0.2,3.9,10.0,-5.0
11,moutarde au moût de raisin,Plastique,"47.833333,-0.333333",en:groceries,8.2,29.0,5.1,18.0
21,Pur jus de pomme,Plastique,"47.833333,-0.333333",en:squeezed-apple-juices,0.0,15.5,0.2,9.0
24,Salade de carottes râpées,Plastique,"47.833333,-0.333333",en:seasoned-grated-carrots,0.3,5.3,0.9,1.0
33,Tarte noix de coco,"Plastique, Barquette","47.833333,-0.333333",en:coconut-pies,22.0,27.3,4.6,14.0


The following columns are unnecessary too, so I drop them too

In [ ]:
world_food_data=world_food_data.drop(['product_name', 'main_category'], axis=1)

Besides, I'd like to split the first packaging coordinates column first_packaging_code_geo into two separate columns - one for the latitude, and one for the longitude for easy plotting later on. I will also round these coordinates to two decimal places and drop the old column.

Finally, I'm going to reset the index column, since I dropped a lot of rows in the previous steps.

In [ ]:
world_food_data[["fp_lat","fp_lon"]]=world_food_data["first_packaging_code_geo"].str.split(",", 1, expand=True)
world_food_data.fp_lat=round(world_food_data.fp_lat.astype(float),2)
world_food_data.fp_lon=round(world_food_data.fp_lon.astype(float),2)
world_food_data=world_food_data.drop(columnUntitled0s="first_packaging_code_geo")

world_food_data.nutrition_score=world_food_data.nutrition_score.astype(int)

world_food_data=world_food_data.reset_index(drop=True)

<ipython-input-17-d7c6eb0c7546>:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  world_food_data[["fp_lat","fp_lon"]]=world_food_data["first_packaging_code_geo"].str.split(",", 1, expand=True)


In [ ]:
world_food_data.packaging=world_food_data.packaging.str.lower()
world_food_data['packaging'] = pd.Categorical(world_food_data['packaging']).codes

In [ ]:
world_food_data.head()

,packaging,fat_g,carbohydrates_g,proteins_g,nutrition_score,fp_lat,fp_lon
0,32697,0.2,3.9,10.0,-5,47.83,-0.33
1,32697,8.2,29.0,5.1,18,47.83,-0.33
2,32697,0.0,15.5,0.2,9,47.83,-0.33
3,32697,0.3,5.3,0.9,1,47.83,-0.33
4,32832,22.0,27.3,4.6,14,47.83,-0.33


In [ ]:
print('LEN OF DATA - ', len(world_food_data))

LEN OF DATA -  958012


In [ ]:
world_food_data.to_csv('cleaned_OFF.csv')

That's all. File cleaned_OPF.csv is used for k-means training